In [1]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None


In [2]:
df01 = pd.read_csv('data070101.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (46,51,65,78,83,97,136,148,160,179,190,227,238,256,265,278,302,325) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#开始洗掉上述人群：df_stdt_qualified

df_qualified_non_tab = df_stdt_non_tablet_user.loc[df_stdt_non_tablet_user['作答时间'] >= 90]
df_qualified_tab = df_stdt_tablet_user.loc[df_stdt_tablet_user['作答时间'] >= 120]

df_stdt_qualified = pd.concat([df_qualified_non_tab, df_qualified_tab])

In [9]:
# 全体：H7 矩阵题 全选同一个选项的直接删除：df_matrix


df_matrix = df_stdt_qualified
counter1 = 0
counter2 = 0


for i in range(len(df_matrix)):
    if counter1 + counter2 < len(df_matrix):
        a = df_matrix.iloc[i , 342]
        b = df_matrix.iloc[i , 343]
        c = df_matrix.iloc[i , 344]
        d = df_matrix.iloc[i , 345]
        e = df_matrix.iloc[i , 346]
        f = df_matrix.iloc[i , 347]
        g = df_matrix.iloc[i , 348]

        answer_list = [a,b,c,d,e,f,g]
        answer_set = set(answer_list)

        if len(answer_set) == 1:
            df_matrix = df_matrix.drop(df_matrix.index[i], inplace = False)
            counter2 += 1
            #print (str(counter2) + "DELETED")
        counter1 += 1
        #print(str(counter1) + "PROGRESS")
    else:
        pass
        #print('done')
            
len(df_matrix)

# output是去除后的样本量


4828

In [10]:
# 多孩用户：A1选择的孩子个数和A3-2选的年级个数不同:fk后缀可删

df_multi = df_matrix.loc[(df_matrix['1.你有孩子吗？（单选）'] == 4)
                         |(df_matrix['1.你有孩子吗？（单选）'] == 5)]
df_multi['孩子数确认'] = df_multi.iloc[:, 12:22].sum(axis=1)

df_multi_2_fk = df_multi.loc[(df_multi['孩子数确认'] != 2) 
                           & (df_multi['1.你有孩子吗？（单选）'] == 4)]

df_multi_3_fk = df_multi.loc[(df_multi['孩子数确认'] <= 2) 
                           & (df_multi['1.你有孩子吗？（单选）'] == 5)]




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [35]:
#核心工具！——跑频次和百分比


def basic_analysis(dataframe, starting_q_col, ending_q_col, counter):
    collector = pd.DataFrame()
    for idx_columns in range(starting_q_col,ending_q_col):
        df2 = dataframe.groupby([dataframe.columns[idx_columns]]).agg({counter:'count'})
        df3 = dataframe[dataframe.columns[idx_columns]].describe().to_frame()
        df4 = (df2 / df3.iloc[0].values[0])
        df5 = pd.merge(df2, df4, how = 'inner', on = dataframe.columns[idx_columns])
        df6 = df5.append(df5.sum().rename('Total'))
        df6['问题'] = df6.index.name
        display(df6)
        collector = pd.concat([collector, df6])
    collector.to_excel('output22.xlsx')